In [38]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [39]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
# import datetime 

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

base_url = 'https://dezede.org/dossiers/archives-opera-comique/data?page='

In [67]:
listings = []

for page in range(1, 98):
    
    print(page)
    
    url = base_url + str(page)
    r  = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html5lib")
    titles = soup.find_all('h3')
    
    for title in titles:
        
        no_show=False
        work = title.get_text().strip()
        
        url = 'https://dezede.org' + title.find('a').get('href')
        r  = requests.get(url, headers=headers)        
        soup = BeautifulSoup(r.text, "html5lib")
        
        try:
            composer = soup.find('h3').find('a').find_all('span')
            composer = composer[1].get('title') + ' ' + composer[0].get_text().strip()
        except:
            composer = ''
        
        try:
            s_url=None
            links = soup.find('table', attrs={'class': 'data-table auto-links'}).find_all('a')
            for link in links:
                if 'evenements/?oeuvre=' in link.get('href') and not s_url:
                    s_url = 'https://dezede.org' + link.get('href')
#             print(s_url)            
            r  = requests.get(s_url, headers=headers)
            soup = BeautifulSoup(r.text, "html5lib")            
        except:
            no_show=True
        
        if no_show==False:            
            try:
                print
                pages = soup.find('ul', attrs={'class': 'pagination pagination-sm'}).find_all('li')[-2].get_text().strip()
                pages = int(pages)
            except:
                pages = 1

            for page in range(1, pages + 1):

                p_url = s_url + '&page={}'.format(page)
#                 print(p_url)
                r  = requests.get(p_url, headers=headers)
                soup = BeautifulSoup(r.text, "html5lib")
                dates = soup.find_all('meta', attrs={'itemprop': 'startDate'})
#                 print(dates)    
                for date in dates:
                    date = date.get('content')
#                     print(date)
                    try:
                        date = datetime.strptime(date, '%Y-%m-%d').date()
                    except:
                        date = date.replace('T', ' ')
                        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S').date()            

                    listing = []
                    listing.append(work)
                    listing.append(composer)        
                    listing.append(date)   
#                     print(listing)
                    listings.append(listing)   
    
#Saving results into dataframe and folder            
import pandas as pd                     
headers = ['work', 'composer', 'date']
df = pd.DataFrame(listings, columns=headers)
df.to_csv('../../data/processed/listings/opera_comique.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
